In [1]:
from andi_datasets.models_phenom import models_phenom
from andi_datasets.utils_videos import transform_to_video
from andi_datasets.utils_trajectories import motion_blur
from helpersMSD import *
from helpersGeneration import *
from helpersPlot import *
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [2]:
# number of time steps per trajectory (frames)
T = 200
# number of trajectories
N = 1
# Length of box (pixels), set to 0 to make them start at 0,0
L = 0 
# diffusion coefficient (pixels^2 / frame)
D = 0.1

In [3]:
trajs, labels = models_phenom().single_state(N, 
                                L = L,
                                T = T,
                                Ds = [D, 0.0], # Mean and variance
                                alphas = 1)
print(trajs.shape,labels.shape)
# Need to reshape generated trajectories because they are in format (T,N,dim), but we want them in (N,T,dim)
trajs = trajs.transpose(1,0,2)
labels = labels.transpose(1,0,2)
print(trajs.shape,labels.shape)
# Labels are now in format (N,T,3): for each particle T times the tuple (alpha, D, state)
print(f"Particle 0 [alpha, Gen_D, state]: {labels[0,0,:]}")


(200, 1, 2) (200, 1, 3)
(1, 200, 2) (1, 200, 3)
Particle 0 [alpha, Gen_D, state]: [1.  0.1 2. ]


In [4]:
nposframe = 10

videos = trajectories_to_video(trajs,nposframe,image_props={"upsampling_factor":10,
                                            #"background_intensity": [0,0],
                                            "particle_intensity": [500,20],
                                            #"resolution": 200e-9
                                              })
print(videos.shape)
play_video(videos[0])

(1, 20, 32, 32)
vmin: 51.0 vmax: 611.0 mean: 107.45


INFO:matplotlib.animation:Animation.save using <class 'matplotlib.animation.HTMLWriter'>


### Comparison of our video creation and andi video creation. Disadvanatge of andi creation is slow but has all particles in same video. Some tweaking of parameters do to in order to have similar noise and Signal to noise ratio

In [ ]:
oversamp_factor = 10
nposframe = oversamp_factor
output_length = T//oversamp_factor
exposure_time = 0.1

MB = motion_blur(output_length = output_length, oversamp_factor = oversamp_factor, exposure_time = exposure_time)

trajsTest, labelsTest = models_phenom().single_state(N, 
                                L = L,
                                T = T,
                                Ds = [D, 0.0], # Mean and variance
                                alphas = 1)
print(trajsTest.shape,labelsTest.shape)
# Need to reshape generated trajectories because they are in format (T,N,dim), but we want them in (N,T,dim)
trajsT = trajsTest.transpose(1,0,2)
labelsT = labelsTest.transpose(1,0,2)
print(trajsT.shape,labelsT.shape)
# Labels are now in format (N,T,3): for each particle T times the tuple (alpha, D, state)
print(f"Particle 0 [alpha, Gen_D, state]: {labelsT[0,0,:]}")

video = transform_to_video(
    trajsTest,
    motion_blur_generator=MB,
    optics_props={ "output_region": [-16, -16, 16, 16],
                  },
    
    particle_props = {
        "particle_intensity": [
            500,
            20,
        ]
    },
    background_props = {
        "background_mean": 50,  # Mean background intensity
        "background_std": 0,  # Standard deviation of background intensity within a video
    }

)
print(video.shape)

videos2= trajectories_to_video(trajsT,nposframe,image_props={"upsampling_factor":5,
                                            "background_intensity": [3500,0],
                                            "particle_intensity": [2000,20]
                                              })
print(videos2.shape)

play_video(normalize_images(video)[0])
play_video(normalize_images(videos2[0])[0])

# Slight difference between the andi-dataset videos and our version.
# The andi version has a much higher background value, we can put our background higher to compensate for this

(200, 1, 2) (200, 1, 3)
(1, 200, 2) (1, 200, 3)
Particle 0 [alpha, Gen_D, state]: [1.  0.1 2. ]


INFO:matplotlib.animation:Animation.save using <class 'matplotlib.animation.HTMLWriter'>


(20, 32, 32, 1)
(1, 20, 32, 32)
vmin: -0.07692252887406889 vmax: 1.0 mean: 0.05


INFO:matplotlib.animation:Animation.save using <class 'matplotlib.animation.HTMLWriter'>


vmin: -0.04408581554889679 vmax: 1.0 mean: 0.08


In [6]:
#%timeit video = transform_to_video(trajsTest,motion_blur_generator=MB,optics_props={ "output_region": [-16, -16, 16, 16]},background_props = {"background_mean": 50,  "background_std": 0, })
trajsTime, _ = models_phenom().single_state(1000, 
                                L = L,
                                T = T,
                                Ds = [D, 0.0], # Mean and variance
                                alphas = 1)
trajsTime = trajsTime.transpose(1,0,2)


%timeit videos = trajectories_to_video(trajs,nposframe,{"upsampling_factor":5,"poisson_noise":-1 })
%timeit videos = trajectories_to_video(trajs,nposframe,{"upsampling_factor":5,"poisson_noise":1  })

#%timeit videos2 = trajectories_to_video2(trajs,nposframe,{"upsampling_factor":5, })

71.1 ms ± 2.54 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
72.8 ms ± 2.77 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Test of center parameter. With centered trajectory the particle is in the center in all Frames. Will be useful if you want to train a model on centered images only

In [7]:
videosCentered= trajectories_to_video(trajsT,nposframe,
                                              center=True,
                                              image_props={"upsampling_factor":5,
                                            "background_intensity": [200,20],
                                            "particle_intensity": [500,20],
                                            "output_size" : 31
                                              })

videosUncentered= trajectories_to_video(trajsT,nposframe,
                                        image_props={"upsampling_factor":5,
                                            "background_intensity": [200,20],
                                            "particle_intensity": [500,20],
                                            "output_size" : 31

                                              })

print(videos.shape)
play_video(videosCentered[0,:,:,:])
play_video(videosUncentered[0,:,:,:])

INFO:matplotlib.animation:Animation.save using <class 'matplotlib.animation.HTMLWriter'>


(1, 20, 32, 32)
vmin: 116.0 vmax: 712.0 mean: 207.90


INFO:matplotlib.animation:Animation.save using <class 'matplotlib.animation.HTMLWriter'>


vmin: 115.0 vmax: 738.0 mean: 207.86


### Test of nomralization value. The theoretical_maximum is the background value + particle intensity. However using a even value of output_size will reduce this value because of the high to low resolution effect 

In [8]:
nposframe = 10

#%timeit video = transform_to_video(trajsTest,motion_blur_generator=MB,optics_props={ "output_region": [-16, -16, 16, 16]},background_props = {"background_mean": 50,  "background_std": 0, })
trajsTest, _ = models_phenom().single_state(1, 
                                L = L,
                                T = T,
                                Ds = [0.1, 0.0], # Mean and variance
                                alphas = 1)
trajsTest = trajsTest.transpose(1,0,2)

#trajsTest = np.zeros((1,T,2))
videos= trajectories_to_video(trajsTest,nposframe,
                                              center=True,
                                              image_props={"upsampling_factor":2,
                                            "background_intensity": [200,10],
                                            "particle_intensity": [500,0],
                                            "poisson_noise":-1,
                                            "output_size":32,

                                              })

play_video(videos[0,:,:,:])
# In general set background mean and std in fct of background_intensity params
# And theoretical_max to background mean + particle_intensity
background_mean, background_std, theoretical_max = 200,10,700
videos_n, vals = normalize_images(videos,background_mean,background_std,theoretical_max)
print(vals)
play_video(videos_n[0,:,:,:])

INFO:matplotlib.animation:Animation.save using <class 'matplotlib.animation.HTMLWriter'>


vmin: 160.22239685058594 vmax: 657.5642700195312 mean: 207.64


INFO:matplotlib.animation:Animation.save using <class 'matplotlib.animation.HTMLWriter'>


(200, 10, 700)
vmin: -0.05838745832443237 vmax: 0.9167926907539368 mean: 0.03


### Test of resolution and trajectory_unit. A resolution of 200nm should give a particle moving half as far as with 100nm resolution  

In [9]:
# number of time steps per trajectory (frames)
T = 200
# number of trajectories
N = 1
# Length of box (pixels), set to 0 to make them start at 0,0
L = 0 
# diffusion coefficient (pixels^2 / s)
D = 5
nposframe = 10

trajs, labels = models_phenom().single_state(N, 
                                L = L,
                                T = T,
                                Ds = [D, 0.0], # Mean and variance
                                alphas = 1)
print(trajs.shape,labels.shape)
# Need to reshape generated trajectories because they are in format (T,N,dim), but we want them in (N,T,dim)
trajs = trajs.transpose(1,0,2)
labels = labels.transpose(1,0,2)
print(trajs.shape,labels.shape)
# Labels are now in format (N,T,3): for each particle T times the tuple (alpha, D, state)
print(f"Particle 0 [alpha, Gen_D, state]: {labels[0,0,:]}")


# convert trajectories of D (pixels/s) to D (micro_m/ms)
trajs = trajs / 100


videosR100 = trajectories_to_video(trajs,nposframe,
                                   center=True,
                                   image_props={"upsampling_factor":10,
                                            "background_intensity": [100,10],
                                            "particle_intensity": [500,20],
                                            "resolution": 100e-9,
                                            "trajectory_unit" : 1000,
                                            "output_size": 7,
                                            "poisson_noise" : -1

                                              })


play_video(videosR100[0])


INFO:matplotlib.animation:Animation.save using <class 'matplotlib.animation.HTMLWriter'>


(200, 1, 2) (200, 1, 3)
(1, 200, 2) (1, 200, 3)
Particle 0 [alpha, Gen_D, state]: [1. 5. 2.]
vmin: 89.68570709228516 vmax: 586.6424560546875 mean: 249.44


# Test of Signal to Noise ratio. What is the max/min snr we can have to still have useful data ? 

In [10]:
videosCentered= trajectories_to_video(trajsT,nposframe,
                                              image_props={"upsampling_factor":5,
                                            "background_intensity": [500,20],
                                            "particle_intensity": [500,20],
                                            "output_size" : 32,
                                            "poisson_noise" : 10
                                              })

videosUncentered= trajectories_to_video(trajsT,nposframe,
                                        image_props={"upsampling_factor":5,
                                            "background_intensity": [0,0],
                                            "particle_intensity": [500,20],
                                            "output_size" : 32,
                                            "poisson_noise" : -1

                                              })

print(videos.shape)
print("Maximum Noise:")
play_video(videosCentered[0,:,:,:])
print("Minimum/No Noise:")
play_video(videosUncentered[0,:,:,:])

INFO:matplotlib.animation:Animation.save using <class 'matplotlib.animation.HTMLWriter'>


(1, 20, 32, 32)
Maximum Noise:
vmin: 422.29998779296875 vmax: 1026.699951171875 mean: 507.52


INFO:matplotlib.animation:Animation.save using <class 'matplotlib.animation.HTMLWriter'>


Minimum/No Noise:
vmin: 0.0 vmax: 467.5299987792969 mean: 7.50
